# Titanic Completed
- a toy project using the Titanic dataset by applying many different techinques of ML into a project
- done for studying purposes
- mainly referencing Machine Learning Pocket Reference(both original text in English published in 2019 and the Korean translation from 2021) & my boot camp materials for studying and coding(Codestates AI Bootcamp)
- https://www.kaggle.com/code/startupsci/titanic-data-science-solutions/notebook

### Error
* profilereport 안됨

## Import Packages & Libraries

In [1]:
import matplotlib.pyplot as plt
import pandas as pd
from sklearn import ensemble, preprocessing, tree
from sklearn.metrics import auc,confusion_matrix, roc_auc_score,roc_curve
from sklearn.model_selection import StratifiedKFold
from sklearn.dummy import DummyClassifier
from yellowbrick.classifier import ConfusionMatrix,ROCAUC
from yellowbrick.model_selection import LearningCurve 
import tensorflow as tf
import tensorflow_datasets as tfds
from pandas_profiling import ProfileReport

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## EDA & Preprocessing of the dataset

In [3]:
train_df = pd.read_csv('/content/drive/MyDrive/dhata/titanic/train.csv')
test_df = pd.read_csv('/content/drive/MyDrive/dhata/titanic/test.csv')
combined = [train_df, test_df] # for some other operations

In [4]:
print(train_df.columns.values)

['PassengerId' 'Survived' 'Pclass' 'Name' 'Sex' 'Age' 'SibSp' 'Parch'
 'Ticket' 'Fare' 'Cabin' 'Embarked']


In [5]:
train_df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


### Clean Data & Create Features

* Before getting started, there's an issue of leaky features(누수, 유출된 특징) in the original dataset(it's the case dealt with in the book). But leaky features of 'boat', 'body' are removed in this dataset.

In [6]:
train_df.dtypes

PassengerId      int64
Survived         int64
Pclass           int64
Name            object
Sex             object
Age            float64
SibSp            int64
Parch            int64
Ticket          object
Fare           float64
Cabin           object
Embarked        object
dtype: object

In [7]:
test_df.dtypes

PassengerId      int64
Pclass           int64
Name            object
Sex             object
Age            float64
SibSp            int64
Parch            int64
Ticket          object
Fare           float64
Cabin           object
Embarked        object
dtype: object

In [8]:
train_df.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [9]:
train_df.isna().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [10]:
test_df.describe()

,PassengerId,Pclass,Age,SibSp,Parch,Fare
count,418.000000,418.000000,332.000000,418.000000,418.000000,417.000000
mean,1100.500000,2.265550,30.272590,0.447368,0.392344,35.627188
std,120.810458,0.841838,14.181209,0.896760,0.981429,55.907576
min,892.000000,1.000000,0.170000,0.000000,0.000000,0.000000
25%,996.250000,1.000000,21.000000,0.000000,0.000000,7.895800
50%,1100.500000,3.000000,27.000000,0.000000,0.000000,14.454200
75%,1204.750000,3.000000,39.000000,1.000000,0.000000,31.500000
max,1309.000000,3.000000,76.000000,8.000000,9.000000,512.329200


In [11]:
test_df.isna().sum()

PassengerId      0
Pclass           0
Name             0
Sex              0
Age             86
SibSp            0
Parch            0
Ticket           0
Fare             1
Cabin          327
Embarked         0
dtype: int64

* Between the two features with many missing values, 'Age' would be important when predicting if a certain person survived or not, but 'Cabin' might not, so we can keep the former and remove the latter.

In [12]:
train_df.Embarked.value_counts(dropna=False) # including missing values when counting

S      644
C      168
Q       77
NaN      2
Name: Embarked, dtype: int64

In [13]:
test_df.Embarked.value_counts(dropna=False)

S    270
C    102
Q     46
Name: Embarked, dtype: int64

In [14]:
train_df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [15]:
test_df.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


* 'Name', 'Ticket', 'Cabin' columns will offer very little importance unless we try to apply some  NLP algorithms successfully. So they should be removed here.

In [16]:
removed = ['Name', 'Ticket', 'Cabin']
train_df = train_df.drop(columns = removed)
test_df = test_df.drop(columns = removed)

In [17]:
train_df.head()

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,1,0,3,male,22.0,1,0,7.2500,S
1,2,1,1,female,38.0,1,0,71.2833,C
2,3,1,3,female,26.0,0,0,7.9250,S
3,4,1,1,female,35.0,1,0,53.1000,S
4,5,0,3,male,35.0,0,0,8.0500,S


In [18]:
train_df = pd.get_dummies(train_df) # convert categorical variable into dummy/indicator variables.
test_df = pd.get_dummies(test_df)

In [19]:
train_df.head()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S
0,1,0,3,22.0,1,0,7.2500,0,1,0,0,1
1,2,1,1,38.0,1,0,71.2833,1,0,1,0,0
2,3,1,3,26.0,0,0,7.9250,1,0,0,0,1
3,4,1,1,35.0,1,0,53.1000,1,0,0,0,1
4,5,0,3,35.0,0,0,8.0500,0,1,0,0,1


In [20]:
test_df.head()

,PassengerId,Pclass,Age,SibSp,Parch,Fare,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S
0,892,3,34.5,0,0,7.8292,0,1,0,1,0
1,893,3,47.0,1,0,7.0000,1,0,0,0,1
2,894,2,62.0,0,0,9.6875,0,1,0,1,0
3,895,3,27.0,0,0,8.6625,0,1,0,0,1
4,896,3,22.0,1,1,12.2875,1,0,0,0,1


* Since 'Sex_female' and 'Sex_male' are very highly correlated, we remove one of them to avoid multicolinearity(다중공선성). 

In [21]:
train_df = train_df.drop(columns = 'Sex_male')
test_df = test_df.drop(columns = 'Sex_male')

In [22]:
train_df.columns

Index(['PassengerId', 'Survived', 'Pclass', 'Age', 'SibSp', 'Parch', 'Fare',
       'Sex_female', 'Embarked_C', 'Embarked_Q', 'Embarked_S'],
      dtype='object')

In [23]:
X_train = train_df.drop(columns = 'Survived')
y_train = train_df.Survived

X_test = test_df

In [24]:
X_train

,PassengerId,Pclass,Age,SibSp,Parch,Fare,Sex_female,Embarked_C,Embarked_Q,Embarked_S
0,1,3,22.0,1,0,7.2500,0,0,0,1
1,2,1,38.0,1,0,71.2833,1,1,0,0
2,3,3,26.0,0,0,7.9250,1,0,0,1
3,4,1,35.0,1,0,53.1000,1,0,0,1
4,5,3,35.0,0,0,8.0500,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...
886,887,2,27.0,0,0,13.0000,0,0,0,1
887,888,1,19.0,0,0,30.0000,1,0,0,1
888,889,3,NaN,1,2,23.4500,1,0,0,1
889,890,1,26.0,0,0,30.0000,0,1,0,0


### Impute Data(데이터 값 대치하기)
* using imputer, on the training dataset first and then on the test dataset so that there's no data leaking
* The IterativeImputer class in scikit-learn supports inductive mode, meaning we can call .fit and then .transform(can impute for new data based on how the model was trained)
>여기서 simpleimputer를 쓰는 것의 장점? 평균값 등을 이용해서 대치를 빠르게 할 수 있다. 하지만 iterative imputer를 사용하면 다른 값들을 이용해서 결측치를 조금 더 합리적으로 예측할 수 있을 것이다.

In [25]:
from sklearn.experimental import enable_iterative_imputer
from sklearn import impute

In [26]:
num_cols =  ["Pclass",'Age','SibSp','Parch','Fare','Sex_female']
imputer = impute.IterativeImputer()
imputed = imputer.fit_transform(X_train[num_cols])
X_train.loc[:,num_cols] = imputed

In [27]:
imputed = imputer.transform(X_test[num_cols])
X_test.loc[:, num_cols] = imputed

* to impute data with the median value, you can simply use pandas
```python
meds = X_train.median()
X_train = X_train.fillna(meds)
X_test = X_test.fillna(meds)
```

### Normalize Data
* normalizing data helps improve many models' performance 

In [31]:
# # the book suggested using these onnly four columns, and I don't understand why I should exclude 'Parch' column when I have to include similar column of 'SibSp'
# also the code is faulty itself. generates an error message
# I will use standardscaler to all the columns and see which columns are effective if that's a matter of choice to use dummy columns
# cols = "Pclass,Age,SibSp,Fare".split(',') # not including dummy columns
# X_train = pd.DataFrame(X_train, columns = cols)

In [34]:
cols = X_train.columns

In [35]:
sca = preprocessing.StandardScaler()
X_train = sca.fit_transform(X_train) # X_train is an array now
X_train = pd.DataFrame(X_train, columns=cols) # making it into a pandas dataframe

In [40]:
X_test = sca.transform(X_test)
X_test = pd.DataFrame(X_test, columns = cols)

/usr/local/lib/python3.7/dist-packages/sklearn/base.py:451: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  "X does not have valid feature names, but"


In [42]:
X_test

,PassengerId,Pclass,Age,SibSp,Parch,Fare,Sex_female,Embarked_C,Embarked_Q,Embarked_S
0,-1.727254,-1.772692,-2.121280,-0.905118,-1.061646,-0.658303,-2.281889,-1.714408,11.263866,-5.221997
1,-1.727239,-1.772692,-2.053988,-0.081855,-1.061646,-0.658640,2.099884,-1.714408,-1.402157,-0.231169
2,-1.727224,-3.204883,-1.973239,-0.905118,-1.061646,-0.657550,-2.281889,-1.714408,11.263866,-5.221997
3,-1.727209,-1.772692,-2.161654,-0.905118,-1.061646,-0.657966,-2.281889,-1.714408,-1.402157,-0.231169
4,-1.727194,-1.772692,-2.188571,-0.081855,0.479188,-0.656496,2.099884,-1.714408,-1.402157,-0.231169
...,...,...,...,...,...,...,...,...,...,...
413,-1.721012,-1.772692,-2.154465,-0.905118,-1.061646,-0.658214,-2.281889,-1.714408,-1.402157,-0.231169
414,-1.720996,-4.637075,-2.097055,-0.905118,-1.061646,-0.617329,2.099884,4.821529,-1.402157,-5.221997
415,-1.720981,-1.772692,-2.099746,-0.905118,-1.061646,-0.658538,-2.281889,-1.714408,-1.402157,-0.231169
416,-1.720966,-1.772692,-2.154465,-0.905118,-1.061646,-0.658214,-2.281889,-1.714408,-1.402157,-0.231169


In [30]:
# X_train = pd.DataFrame(X_train)
# X_train

### Refactor

In [ ]:
def tweak_titanic(df):
  df = df.drop(
      columns = [
          'name', 'ticket', 'cabin'
      ]
  ).pipe(pd.get_dummies, drop_first = True)

## Baseline Model

In [44]:
bm = DummyClassifier()
bm.fit(X_train, y_train)
bm.score(X_train, y_train) # since there's no y_test

0.6161616161616161

## Various Algorithms

In [50]:
from sklearn import model_selection
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB 
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
import xgboost
for model in [
    DummyClassifier,
    LogisticRegression,
    DecisionTreeClassifier,
    KNeighborsClassifier,
    GaussianNB,
    SVC,
    RandomForestClassifier,
    xgboost.XGBClassifier
    ]:
    cls = model()
    kfold = model_selection.KFold(
        n_splits = 10
    )
    s = model_selection.cross_val_score(
        cls,X_train, y_train, scoring = 'roc_auc', cv = kfold
    )
    print(f"{model.__name__:22} AUC: "
    f"{s.mean():.3f} STD: {s.std():.2f}"
    )



DummyClassifier        AUC: 0.500 STD: 0.00
LogisticRegression     AUC: 0.851 STD: 0.03
DecisionTreeClassifier AUC: 0.736 STD: 0.06
KNeighborsClassifier   AUC: 0.837 STD: 0.04
GaussianNB             AUC: 0.815 STD: 0.04
SVC                    AUC: 0.854 STD: 0.04
RandomForestClassifier AUC: 0.859 STD: 0.04
XGBClassifier          AUC: 0.863 STD: 0.04


## Stacking
* predicts the labels taking other models and using the outputs
* could use when you want maximum performance at the cost of interpretability

In [55]:
# import six
# import sys
# sys.modules['sklearn.externals.six'] = six

# # handling the error message of "ImportError: cannot import name 'six' from 'sklearn.externals' "

In [60]:
# from mlxtend.classifier import StackingClassifier


# clfs = [
#     x()
#     for x in [
#         LogisticRegression,
#         DecisionTreeClassifier,
#         KNeighborsClassifier,
#         GaussianNB,
#         SVC,
#         RandomForestClassifier
#     ]
# ]

# stack = StackingClassifier(
#     classifiers = clfs,
#     meta_classifier = LogisticRegression()

# )

# kfold = model_selection.KFold(
#     n_splits = 10
# )

# s = model_selection.cross_val_score(
#     stack, X_train, y_train, scoring = 'roc_auc', cv = kfold
# )

# print(
#     f"{stack.__class__.__name__}"
#     f"AUC: {s.mean():.3f} STD: {s.std():.2f}"
# )

# """
# error messages:
# AttributeError: 'StackingClassifier' object has no attribute 'decision_function'
# AttributeError: 'StackingClassifier' object has no attribute 'classes_'
# """

/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py:774: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_scorer.py", line 355, in _score
    y_pred = method_caller(clf, "decision_function", X)
  File "/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_scorer.py", line 68, in _cached_call
    return getattr(estimator, method)(*args, **kwargs)
AttributeError: 'StackingClassifier' object has no attribute 'decision_function'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py", line 761, in _score
    scores = scorer(estimator, X_test, y_test)
  File "/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_scorer.py", line 103, in __call__
    score = scor

StackingClassifierAUC: nan STD: nan


/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py:774: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_scorer.py", line 355, in _score
    y_pred = method_caller(clf, "decision_function", X)
  File "/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_scorer.py", line 68, in _cached_call
    return getattr(estimator, method)(*args, **kwargs)
AttributeError: 'StackingClassifier' object has no attribute 'decision_function'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py", line 761, in _score
    scores = scorer(estimator, X_test, y_test)
  File "/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_scorer.py", line 103, in __call__
    score = scor

## Create Model

In [65]:
rf = ensemble.RandomForestClassifier(n_estimators = 100, random_state = 42)
rf.fit(X_train,y_train)
RandomForestClassifier(bootstrap = True, class_weight = None, criterion = 'gini', max_depth = None, max_features =' auto', max_leaf_nodes = None, min_impurity_decrease = 0.0,
 min_samples_leaf = 1, min_samples_split=2, min_weight_fraction_leaf = 0.0, n_estimators = 10, n_jobs = 1, oob_score = False, random_state=42, verbose = 0, warm_start = False)

RandomForestClassifier(max_features=' auto', n_estimators=10, n_jobs=1,
                       random_state=42)